# Comic Crafter with LocalTunnel

This notebook demonstrates how to run the Comic Crafter application in Google Colab using LocalTunnel for port exposure.

In [ ]:
# Install Git LFS if needed and clone the repository
!git clone https://github.com/[your-username]/comic-crafter.git
%cd comic-crafter

In [ ]:
# Install Node.js and localtunnel
!curl -fsSL https://deb.nodesource.com/setup_18.x | sudo -E bash -
!sudo apt-get install -y nodejs
!npm install -g localtunnel

# Install Python dependencies
!pip install -r server/requirements.txt

In [ ]:
# Setup the base model with LORA weights
%cd utils
!python setup_sd.py
%cd ..

In [ ]:
# Start the backend server in background
import subprocess
import time
import os

# Start server in background
server_process = subprocess.Popen(["python", "server/app.py"])

# Wait for server to start
time.sleep(5)
print(f"Server started with PID: {server_process.pid}")

In [ ]:
# Expose the server with localtunnel
# Note: This command will print the public URL to the console
!lt --port 5000

In [ ]:
# Optional: Setup Ollama for additional LLM capabilities
# Install lshw first
!apt-get install -y lshw

# Install Ollama
!curl -fsSL https://ollama.ai/install.sh | sh

# Start Ollama in background with nohup
!nohup ollama serve > ollama.log 2>&1 &
time.sleep(5)
print("Ollama server started in background")

# Pull a model in background
!nohup ollama pull llama3 > pull_llama3.log 2>&1 &
print("Started pulling llama3 model in background")

In [ ]:
# Build the frontend
%cd client
!npm install
!npm run build
%cd ..

In [ ]:
# Optionally serve the frontend with localtunnel as well
import http.server
import socketserver
from threading import Thread

# Serve the built frontend
PORT_FRONTEND = 3000
DIRECTORY = "client/dist"

class Handler(http.server.SimpleHTTPRequestHandler):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, directory=DIRECTORY, **kwargs)

def start_frontend_server():
    with socketserver.TCPServer(("", PORT_FRONTEND), Handler) as httpd:
        httpd.serve_forever()

# Start frontend server in background
frontend_thread = Thread(target=start_frontend_server, daemon=True)
frontend_thread.start()
print(f"Frontend server started on port {PORT_FRONTEND}")

# Expose frontend with localtunnel
print("Exposing frontend with localtunnel...")
!lt --port 3000

## Notes:

1. Make sure to copy the localtunnel URLs that appear in the output above
2. The backend API URL will be used in the frontend configuration
3. After kernel restart, you'll need to run these cells again
4. LocalTunnel URLs are temporary and will change each time you restart the tunnel